In [ ]:
%load_ext autoreload
%autoreload 2

import spikeinterface as si
import spikewidgets as sw
import spiketoolkit as st

from kbucket import client as kb
from pairio import client as pa

from spikeforest_sort import spikeforest_sort, mountainsort4b_params

from sortingcomparison import SortingComparison

In [ ]:
pa.setConfig(collections=['magland'])
kb.setConfig(share_ids=['magland.eh1'])

In [ ]:
pa.setConfig(user=pa.getLocal('pairio-user'),token=pa.getLocal('pairio-token'))
kb.setConfig(upload_share_id='magland.eh1',upload_token=pa.getLocal('eh1-token'))

In [ ]:
sorters=[]
ms4_sorter=st.sorters.mountainsort4
setattr(ms4_sorter,'name','mountainsort4')
setattr(ms4_sorter,'version','4.0.0')
ms4_params=mountainsort4b_params(adjacency_radius=-1,detect_sign=-1,detect_threshold=3)
sorters.append(dict(
    name='mountainsort4',
    sorter=ms4_sorter,
    params=ms4_params
))

In [ ]:
datasets=[]
study_dir='kbucket://b5ecdf1474c5/spikeforest/gen_synth_datasets/datasets_noise10_K10'
dd=kb.readDir(study_dir)
for dsname in dd['dirs']:
    dsdir='{}/{}'.format(study_dir,dsname)
    datasets.append(dict(
        name=dsname,
        dsdir=dsdir
    ))

In [ ]:
dd=kb.readDir(study_dir)
results=[]
for dataset in datasets:
    for sorter in sorters:
        print('SORTER={}, DATASET={}'.format(sorter['name'],dataset['name']))
        sorting=spikeforest_sort(
            recording_dirname=dataset['dsdir'],
            sorter=sorter['sorter'],
            sorting_params=sorter['params'],
            _force_run=False,
            _force_save=False
        )
        results.append(dict(
            sorter=sorter,
            dataset=dataset,
            sorting=sorting
        ))

In [ ]:
result=results[0]

In [ ]:
dsdir=result['dataset']['dsdir']
recording=si.MdaRecordingExtractor(dataset_directory=dsdir,download=False)
sorting_true=si.MdaSortingExtractor(firings_file=dsdir+'/firings_true.mda')
sorting=result['sorting']

In [ ]:
SC=SortingComparison(sorting_true,sorting)

In [ ]:
sw.SortingComparisonTable(comparison=SC).display()